In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-jul-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-jul-2021/test.csv")

In [ ]:
train_labels = train.iloc[:,-3:]  #要素抽出
features = pd.concat([train.iloc[:,1:-3], test.iloc[:,1:]])

In [ ]:
from sklearn.preprocessing import StandardScaler  #データセットを標準化
scaler = StandardScaler()
scaled_features =scaler.fit_transform(features)
scaled_train = scaled_features[:train.shape[0]]
scaled_test = scaled_features[train.shape[0]:]

In [ ]:
from sklearn.model_selection import train_test_split #訓練データとテストデータに分割
x_train, x_test, y_train, y_test = train_test_split(scaled_train, train_labels, random_state = 42, test_size = 0.3)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

#シーケンシャルモデルを生成
model = keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=[8]),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation = 'relu'),
    layers.Dense(32, activation = 'relu'),
    layers.Dense(3),
])

In [ ]:
model.compile(
optimizer = "adam", #最適化関数
loss = "mae") #平均的絶対誤差(予測値と正解値の差)

#モデルの学習
history = model.fit(
x_train, y_train,
validation_data=(x_test, y_test),
batch_size = 256, #過学習を防ぐため...
epochs = 50, #フィティング回数
verbose = False)

In [ ]:
hstry_df = pd.DataFrame(history.history)
hstry_df["loss"].plot() #データを可視化

In [ ]:
submissions = model.predict(scaled_test) #提出データ
submissions
submission_df = pd.DataFrame(submissions, columns = train_labels.columns)
submission_df['date_time'] = test['date_time']
submission_df.to_csv("submissions.csv", index=False)